## Retriever And Chain With Langchain

In [13]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("tourism.pdf")
docs = loader.load()
docs

[Document(metadata={'source': 'tourism.pdf', 'page': 0, 'page_label': '1'}, page_content='MODULE – 4\n 69\nNatural Tourist Attractions in India\nTOURISM\nNatural Diversity as\nTourist Attraction\nNotes\n12\nNATURAL TOURIST\nATTRACTIONS IN INDIA\nIndia is a vast country with immense geographical and cultural diversity. The\ndiversity varies from the Himalayan Mountain in north to coasts in the south\nand from Thar Desert of Rajasthan in the west to the humid forests of the North-\nEast. Even the climatic conditions vary from the equatorial in the south to the\npolar region in the higher slope of northern mountain. This rich diversity is bound\nto create many natural attractions for the tourists visiting these places. The\npeople from plain and Southern India would like to visit the Himalayan hill\nstations in search of a cool climate during summer. Similarly, people from the\nnorth would like to see the coastal areas. This plays a great role in providing\nopportunities for the developme

In [14]:
from langchain_community.embeddings import HuggingFaceEmbeddings
## FAISS Vector Database
from langchain_community.vectorstores import FAISS


# Use Hugging Face embeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

db_f = FAISS.from_documents(docs[:15], embeddings)

print("Embedding complete!")

Embedding complete!


In [15]:
query="places in deccan plateau"
result=db_f.similarity_search(query)
result[0].page_content

'MODULE – 4\n 71\nNatural Tourist Attractions in India\nTOURISM\nNatural Diversity as\nTourist Attraction\nNotes\nFigure 12.1: Physical Features of India\nThe British rulers in India developed some of these hill stations to escape the\nheat of the summer seasons. These hill stations are very much in demand even\ntoday, particularly in the summer season, to get respite from the scorching heat\nof the sun in the plains and plateau region of the country.\nOuter Himalaya or the Siwalik: They are the southern most range of the\nmountains and are popularly known as the foothills of the Himalayas. Its altitude\nvaries from 900 to 1500 meter and its width from 15 to 50 km. They are the\nnewly formed ranges of the Himalayas. To the south of this range are the great\nplains of North India. The adjoining land between plains and the mountains is\nknown as the Terai region. It has very dense vegetation which is deciduous in\nnature and is characterized by many long, flat-bottomed valleys known as D

In [16]:
from langchain_community.llms import Ollama
## Load Ollama LAMA2 LLM model
llm=Ollama(model="mistral")
llm

Ollama(model='mistral')

In [17]:
## Design ChatPrompt Template
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""Hey there! I need your help answering a question based only on the provided context. Please carefully analyze the information step by step before providing a clear and well-structured response. Stick strictly to the given details—no outside knowledge!

Here's the context you should use:
<context>
{context}
</context>

Now, here's the question:
{input}

Take your time and ensure the answer is as accurate and helpful as possible!""")

In [18]:
## Chain Introduction
## Create Stuff Docment Chain

from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain=create_stuff_documents_chain(llm,prompt)

In [19]:
"""
Retrievers: A retriever is an interface that returns documents given
 an unstructured query. It is more general than a vector store.
 A retriever does not need to be able to store documents, only to 
 return (or retrieve) them. Vector stores can be used as the backbone
 of a retriever, but there are other types of retrievers as well. 
 https://python.langchain.com/docs/modules/data_connection/retrievers/   
"""
retriever=db_f.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x78f85cf6ff80>, search_kwargs={})

In [20]:
"""
Retrieval chain:This chain takes in a user inquiry, which is then
passed to the retriever to fetch relevant documents. Those documents 
(and original inputs) are then passed to an LLM to generate a response
https://python.langchain.com/docs/modules/chains/
"""

from langchain.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(retriever,document_chain)

In [24]:
response=retrieval_chain.invoke({"input":"Where is Mount Abu situated and places to visit"})

In [25]:
response['answer']

" Mount Abu is located in the Aravalli Range of Rajasthan, in western India. It serves as a popular hill station due to its cooler climate compared to the surrounding plains. Here are some places to visit when you're there:\n\n1. Nakki Lake: This artificial lake is one of the primary attractions at Mount Abu, and it offers boat rides to visitors. You can also find shopping centers around the lake that sell local handicrafts, jewelry, and other items.\n2. Dilwara Temples: Located 2.5 km from Nakki Lake, these beautiful Jain temples date back to the 11th and 13th centuries. Their intricate marble carvings make them a must-visit site for anyone touring Mount Abu.\n3. Sunset Point: As the name suggests, Sunset Point offers stunning sunset views over the surrounding valleys. It's one of the best places to enjoy the natural beauty of Mount Abu during the evening hours.\n4. Guru Shikhar: At an elevation of 1,722 meters (5,680 feet), Guru Shikhar is the highest peak in the Aravalli Range. It o